<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b80e7dcc030a95498bc482ffd4c246b3608a564f5831d7a04c7d2b01e168d44b
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-08 13:06:14
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: -56.71 K (-0.4%)
Current PnL: -21.09 L (-14.0%)
CY Booked + Current PnL: -9.65 L (-6.4%)
-------------------
Total profit:  1.41 L
Total loss:  -22.50 L
-------------------
Total Booked + Current PnL: 17.33 L (14.01%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.14%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.60 L (63.06%)
Deployed:  1.24 C
Current:  1.40 C
CAGR/XIRR %: 7.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.20,119033.0,18529.0,10582.0,3.46,18.44,8.89,28.96,79.0,1.75,0.85,58.66,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.60,39.0,M-SC,1.18,83382.0,-17395.0,17485.0,0.02,-17.26,20.97,0.09,245.0,-0.99,0.60,9.54,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.48,48.0,H-MC,3.04,189100.0,24430.0,18891.0,-1.11,14.84,9.99,26.31,89.0,1.29,1.35,24.00,X40N,NTT,AC
19,CIPLA,1662.74,-22.87,39.0,H-LC,3.81,205459.0,959.0,22333.0,-0.90,0.47,10.87,11.39,10.0,0.04,1.47,9.46,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.66,51.0,H-LC,3.33,216029.0,5663.0,23115.0,0.71,2.69,10.70,13.68,37.0,0.24,1.54,19.66,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-53.52,49.0,H-SC,9.15,129585.0,6249.0,125373.0,4.19,5.07,96.75,106.72,119.0,0.05,0.93,29.63,AR,ATH,MISC
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.20,119033.0,18529.0,10582.0,3.46,18.44,8.89,28.96,79.0,1.75,0.85,58.66,MH,ATH,METALS
40,INFY,2275.00,-18.52,52.0,H-LC,8.01,317285.0,4397.0,167304.0,2.13,1.41,52.73,54.87,3.0,0.03,2.27,8.15,X40,BTT,IT
48,LTIM,7230.20,-2.60,59.0,H-LC,2.46,241448.0,-4703.0,83903.0,1.83,-1.91,34.75,32.18,16.0,-0.06,1.73,33.44,X200,ATH,IT
30,HCLTECH,1937.36,-0.22,55.0,H-LC,9.78,228262.0,-13662.0,75897.0,1.43,-5.65,33.25,25.73,8.0,-0.18,1.63,7.87,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.1,-42.78,38.0,H-SC,16.00,86740.0,-15760.0,56364.0,-3.09,-15.38,64.98,39.61,114.0,-0.28,0.62,5.94,AR,ATH,MISC
53,QUESS,986.0,-46.56,28.0,M-SC,37.78,52845.0,-12161.0,163090.0,-2.90,-18.71,308.62,232.18,198.0,-0.07,0.38,0.00,XY24,NTT,MISC
64,SHALBY,327.0,1195.06,58.0,M-SC,21.36,176057.0,-5086.0,50546.0,-2.66,-2.81,28.71,25.10,235.0,-0.10,1.26,41.09,XY24,NTT,HEALTHCARE
75,TITAGARH,1548.0,-3.20,53.0,H-SC,4.75,206705.0,-35472.0,146244.0,-2.23,-14.65,70.75,45.74,158.0,-0.24,1.48,34.44,XY24,NTT,ENGINEERING
10,BANDHANBNK,400.0,-13.99,45.0,H-SC,2.94,215887.0,-62672.0,320117.0,-2.04,-22.50,148.28,92.42,151.0,-0.20,1.54,24.77,XY24,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.0,43.54,49.0,M-MC,12.46,226357.0,1395.0,166259.0,-1.08,0.62,73.45,74.53,192.0,0.01,1.62,33.07,XY24,BTT,STEEL
35,ICICIPRULI,790.0,-21.68,45.0,H-MC,1.90,134741.0,-1047.0,43804.0,-1.21,-0.77,32.51,31.48,107.0,-0.02,0.96,11.25,X40,ATH,INSURANCE
86,WIPRO,420.0,-13.79,47.0,M-LC,5.87,151497.0,552.0,108896.0,0.34,0.37,71.88,72.51,53.0,0.01,1.08,6.41,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,61.0,H-SC,13.45,129546.0,-12933.0,32451.0,-0.91,-9.08,25.05,13.70,163.0,-0.40,0.93,52.90,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.60,39.0,M-SC,1.18,83382.0,-17395.0,17485.0,0.02,-17.26,20.97,0.09,245.0,-0.99,0.60,9.54,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,48.0,H-SC,2.01,220950.0,-48717.0,85044.0,-0.40,-18.07,38.49,13.47,138.0,-0.57,1.58,12.35,XY24,NTT,PAINTS
18,CERA,9475.0,-23.39,34.0,H-SC,1.61,139552.0,-36351.0,78372.0,-0.35,-20.67,56.16,23.89,149.0,-0.46,1.00,20.52,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-23.71,47.0,H-MC,6.45,103294.0,-28541.0,70767.0,0.87,-21.65,68.51,32.03,98.0,-0.40,0.74,15.70,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,49.0,M-MC,12.46,226357.0,1395.0,166259.0,-1.08,0.62,73.45,74.53,192.0,0.01,1.62,33.07,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,68.0,M-LC,6.90,213911.0,8835.0,103725.0,0.38,4.31,48.49,54.89,52.0,0.09,1.53,29.44,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.48,48.0,H-MC,3.04,189100.0,24430.0,18891.0,-1.11,14.84,9.99,26.31,89.0,1.29,1.35,24.00,X40N,NTT,AC
84,VOLTAS,1530.00,-2.20,48.0,H-MC,1.29,205230.0,13488.0,24279.0,-0.29,7.03,11.83,19.69,99.0,0.56,1.47,14.31,XY25,NTT,AC
17,CAMS,5211.76,-6.71,43.0,H-SC,1.74,105980.0,3976.0,39954.0,-0.41,3.90,37.70,43.06,122.0,0.10,0.76,21.68,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.20,119033.0,18529.0,10582.0,3.46,18.44,8.89,28.96,79.0,1.75,0.85,58.66,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,63.0,H-MC,11.14,191481.0,11673.0,100738.0,-1.38,6.49,52.61,62.52,88.0,0.12,1.37,42.79,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,6.90,213911.0,8835.0,103725.0,0.38,4.31,48.49,54.89,52.0,0.09,1.53,29.44,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.52,49.0,H-SC,9.15,129585.0,6249.0,125373.0,4.19,5.07,96.75,106.72,119.0,0.05,0.93,29.63,AR,ATH,MISC
86,WIPRO,420.00,-13.79,47.0,M-LC,5.87,151497.0,552.0,108896.0,0.34,0.37,71.88,72.51,53.0,0.01,1.08,6.41,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.05,28.0,H-LC,10.18,276892.0,-38950.0,151626.0,-1.36,-12.33,54.76,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.68,30.0,H-LC,5.71,205181.0,-46587.0,99800.0,-0.90,-18.50,48.64,21.13,27.0,-0.47,1.47,10.37,X40,ATH,PAINTS
19,CIPLA,1662.74,-22.87,39.0,H-LC,3.81,205459.0,959.0,22333.0,-0.90,0.47,10.87,11.39,10.0,0.04,1.47,9.46,X40N,ATH,PHARMA
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195135.0,-5003.0,25894.0,-0.19,-2.50,13.27,10.44,4.0,-0.19,1.39,2.92,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-10.67,40.0,H-LC,8.01,250960.0,-6103.0,41233.0,-0.30,-2.37,16.43,13.67,24.0,-0.15,1.79,17.55,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.39,54.0,H-LC,1.96,161265.0,-24830.0,72311.0,-0.78,-13.34,44.84,25.51,15.0,-0.34,1.15,18.84,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,40.0,H-LC,2.33,195135.0,-5003.0,25894.0,-0.19,-2.50,13.27,10.44,4.0,-0.19,1.39,2.92,X40,NTT,FMCG
48,LTIM,7230.2,-2.60,59.0,H-LC,2.46,241448.0,-4703.0,83903.0,1.83,-1.91,34.75,32.18,16.0,-0.06,1.73,33.44,X200,ATH,IT
51,NESTLEIND,1377.0,-10.64,49.0,H-LC,2.76,275090.0,9664.0,47123.0,-0.10,3.64,17.13,21.40,11.0,0.21,1.97,10.53,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,51.0,H-LC,3.33,216029.0,5663.0,23115.0,0.71,2.69,10.70,13.68,37.0,0.24,1.54,19.66,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.05,28.0,H-LC,10.18,276892.0,-38950.0,151626.0,-1.36,-12.33,54.76,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195135.0,-5003.0,25894.0,-0.19,-2.50,13.27,10.44,4.0,-0.19,1.39,2.92,X40,NTT,FMCG
74,TCS,4406.34,-29.97,44.0,H-LC,12.18,279528.0,-66428.0,134677.0,-0.49,-19.20,48.18,19.73,1.0,-0.49,2.00,2.95,X40,ATH,IT
80,UNITDSPR,1644.00,-12.23,52.0,H-LC,6.77,229998.0,-4146.0,52762.0,-1.08,-1.77,22.94,20.77,86.0,-0.08,1.64,5.03,X40N,NTT,BREWERIES
1,ABB,7934.00,-40.25,47.0,H-LC,7.52,249648.0,-11971.0,131190.0,-0.35,-4.58,52.55,45.57,7.0,-0.09,1.78,6.23,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.39,54.0,H-LC,1.96,161265.0,-24830.0,72311.0,-0.78,-13.34,44.84,25.51,15.0,-0.34,1.15,18.84,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195135.0,-5003.0,25894.0,-0.19,-2.50,13.27,10.44,4.0,-0.19,1.39,2.92,X40,NTT,FMCG
19,CIPLA,1662.74,-22.87,39.0,H-LC,3.81,205459.0,959.0,22333.0,-0.90,0.47,10.87,11.39,10.0,0.04,1.47,9.46,X40N,ATH,PHARMA
5,ASIANPAINT,3465.66,-18.68,30.0,H-LC,5.71,205181.0,-46587.0,99800.0,-0.90,-18.50,48.64,21.13,27.0,-0.47,1.47,10.37,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,51.0,H-LC,3.33,216029.0,5663.0,23115.0,0.71,2.69,10.70,13.68,37.0,0.24,1.54,19.66,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.97,44.0,H-LC,12.18,279528.0,-66428.0,134677.0,-0.49,-19.20,48.18,19.73,1.0,-0.49,2.00,2.95,X40,ATH,IT
40,INFY,2275.00,-18.52,52.0,H-LC,8.01,317285.0,4397.0,167304.0,2.13,1.41,52.73,54.87,3.0,0.03,2.27,8.15,X40,BTT,IT
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195135.0,-5003.0,25894.0,-0.19,-2.50,13.27,10.44,4.0,-0.19,1.39,2.92,X40,NTT,FMCG
83,VBL,671.64,-23.05,28.0,H-LC,10.18,276892.0,-38950.0,151626.0,-1.36,-12.33,54.76,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,47.0,H-LC,7.52,249648.0,-11971.0,131190.0,-0.35,-4.58,52.55,45.57,7.0,-0.09,1.78,6.23,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,43.0,L-SC,30.78,78643.0,-34906.0,74955.0,-1.02,-30.74,95.31,35.27,268.0,-0.47,0.56,92.48,XR,NTT,HOTELS
49,MASFIN,398.61,-18.25,49.0,H-SC,10.93,92700.0,-5280.0,26883.0,-0.08,-5.39,29.00,22.05,152.0,-0.20,0.66,34.75,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.20,119033.0,18529.0,10582.0,3.46,18.44,8.89,28.96,79.0,1.75,0.85,58.66,MH,ATH,METALS
37,INDIAMART,4810.62,-53.52,49.0,H-SC,9.15,129585.0,6249.0,125373.0,4.19,5.07,96.75,106.72,119.0,0.05,0.93,29.63,AR,ATH,MISC
78,UJJIVANSFB,60.00,105.37,61.0,H-SC,13.45,129546.0,-12933.0,32451.0,-0.91,-9.08,25.05,13.70,163.0,-0.40,0.93,52.90,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,13.51,144630.0,-4590.0,148766.0,-1.47,-3.08,102.86,96.62,208.0,-0.03,1.03,56.52,XY25,NTT,FINANCE
69,SURYODAY,240.00,60.09,61.0,H-SC,7.36,147880.0,-31191.0,96921.0,-0.19,-17.42,65.54,36.71,135.0,-0.32,1.06,46.73,XR,NTT,BANKS
26,GREENPANEL,537.00,237.21,62.0,M-SC,5.11,150548.0,-32530.0,111511.0,0.82,-17.77,74.07,43.14,230.0,-0.29,1.08,41.61,XY24,NTT,MISC
32,HINDZINC,730.22,31.07,68.0,M-LC,6.90,213911.0,8835.0,103725.0,0.38,4.31,48.49,54.89,52.0,0.09,1.53,29.44,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,63.0,H-MC,11.14,191481.0,11673.0,100738.0,-1.38,6.49,52.61,62.52,88.0,0.12,1.37,42.79,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.12
1,25,43.41
2,50,75.09


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.10
LC    32.20
MC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.99
X40      14.65
X40N     11.16
XR       10.95
XY25     10.39
AR        8.17
OX40N     7.72
X200      1.73
MH        1.61
X5K       1.53
SR        1.13
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.03
H-LC    25.69
H-MC    20.49
M-SC    13.66
M-LC     5.50
M-MC     2.94
L-SC     1.41
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.73,-6.83,42.04
IT,12.78,-17.65,81.37
FINANCE,10.54,-14.27,69.45
MISC,6.95,-18.42,84.04
BANKS,6.31,-13.10,72.60
PAINTS,5.52,-19.71,37.56
ELECTRICAL,5.39,-9.21,47.52
INSURANCE,3.82,-3.49,42.18
AC,3.51,1.41,15.09


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3239578.0,22
XR,1379400.0,14
AR,1185233.0,9
X40,791450.0,10
X40N,673362.0,9
OX40N,569011.0,10
XY25,482957.0,7
SR,263318.0,2
X5K,103725.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3733487.0,29
M-SC,1522458.0,17
H-MC,1287340.0,15
H-LC,1198909.0,15
M-LC,379823.0,4
M-MC,371827.0,2
L-SC,263183.0,3
L-MC,60108.0,1
L-LC,42618.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1201811.0      6
M-SC       XY24       853241.0      7
H-SC       AR         812239.0      5
           XR         804126.0      7
H-MC       XY24       575756.0      4
H-LC       X40        503572.0      5
M-MC       XY24       371827.0      2
H-SC       X40N       311283.0      4
           OX40N      263476.0      4
           SR         263318.0      2
H-LC       X40N       226721.0      3
H-MC       X40        214037.0      4
H-LC       AR         203501.0      2
H-MC       XY25       180102.0      2
L-SC       XR         174141.0      2
M-SC       AR         169493.0      2
M-LC       XY24       167202.0      2
M-SC       XY25       148766.0      1
           OX40N      145726.0      4
H-MC       X40N       135358.0      2
M-SC       XR         131391.0      2
H-LC       XY25       111471.0      3
M-LC       XR         108896.0      1
           X5K        103725.0      1
H-MC       XR         100738.0      1
L-SC       OX40N       89042.0      1
H-LC       X200        83903.0      1
H-SC       MH          77234.0      1
M-SC       X40         73841.0      1
H-MC       OX40N       70767.0      1
H-LC       XY24        69741.0      1
L-MC       XR          60108.0      1
L-LC       XY25        42618.0      1
H-MC       MH          10582.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
